In [56]:
import pandas as pd
import plotly.express as px
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [57]:
dados = pd.read_csv('../dados_modelo_defeitos_dia.csv')
df = pd.DataFrame(dados)

In [58]:
df.head().round(2)

,Ordem de prod,Ref. do Artigo,Data de prod,Temperatura (F°),Peso médio (g),Comprimento médio (mm),Diâmetro,Área da gota (mm),Gotas cortadas (toneladas),Total de Frascos no Dia (unidade),...,Lascado em produção,Dobra,Trinca por choque térmico,Racho na costura,Racho na terminação,Cava no ombro,Ombro mal cheio,Ombro deformado,Martelado,Aderido
0,192970,C0516.0000R,2024-01-03,2126.97,169.3,110.4,32.5,3588.00,20.95,123740.0,...,0,10,5,0,1,10,0,8,0,0
1,192970,C0516.0000R,2024-01-04,2118.92,169.7,85.6,18.1,1549.36,21.02,124294.0,...,1,11,14,0,6,6,4,5,0,0
2,192970,C0516.0000R,2024-01-05,2125.00,173.6,140.5,32.9,4622.45,21.45,126414.0,...,0,0,16,0,3,13,26,11,0,0
3,192970,C0516.0000R,2024-01-06,2125.40,170.2,144.7,32.7,4731.69,21.53,126858.0,...,3,7,20,0,5,2,4,6,0,0
4,192970,C0516.0000R,2024-01-07,2126.76,171.7,141.5,30.2,4273.30,21.44,126856.0,...,1,10,8,0,0,7,2,10,2,0


In [59]:
df.columns

Index(['Ordem de prod', 'Ref. do Artigo', 'Data de prod', 'Temperatura (F°)',
       'Peso médio (g)', 'Comprimento médio (mm)', 'Diâmetro',
       'Área da gota (mm)', 'Gotas cortadas (toneladas)',
       'Total de Frascos no Dia (unidade)', 'Total de Defeitos no Dia',
       'Onda no fundo', 'Terminação deformada', 'Racho no anel',
       'Sujo de óleo', 'Cascão', 'Frasco caído na esteira', 'Dupla costura',
       'Frio', 'Lascado em produção', 'Dobra', 'Trinca por choque térmico',
       'Racho na costura', 'Racho na terminação', 'Cava no ombro',
       'Ombro mal cheio', 'Ombro deformado', 'Martelado', 'Aderido'],
      dtype='object')

## Modelo preditivo

In [60]:
# Variáveis independentes (medidas)
X = df[['Peso médio (g)', 'Comprimento médio (mm)', 'Diâmetro']]

# Variáveis dependentes (defeitos)
y = df[['Aderido', 'Racho na costura', 'Martelado', 'Frio', 'Ombro deformado', 'Racho no anel', 
        'Frasco caído na esteira', 'Ombro mal cheio', 'Dobra', 'Cava no ombro', 'Racho na terminação',
        'Sujo de óleo', 'Trinca por choque térmico', 'Lascado em produção', 'Dupla costura', 'Cascão',
        'Terminação deformada', 'Onda no fundo']]

In [61]:
from sklearn.model_selection import train_test_split

# Dividindo os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [62]:
from sklearn.preprocessing import StandardScaler

# Normalizando os dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Repetir o treinamento com os dados normalizados
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [63]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

model = LinearRegression()

defeitos_preditos = {}
for defeito in y.columns:
    model.fit(X_train, y_train[defeito])
    
    y_pred = model.predict(X_test)
    
    mse = mean_squared_error(y_test[defeito], y_pred)
    r2 = r2_score(y_test[defeito], y_pred)
    
    defeitos_preditos[defeito] = {'MSE': mse, 'R2': r2}

pd.DataFrame(defeitos_preditos)

,Aderido,Racho na costura,Martelado,Frio,Ombro deformado,Racho no anel,Frasco caído na esteira,Ombro mal cheio,Dobra,Cava no ombro,Racho na terminação,Sujo de óleo,Trinca por choque térmico,Lascado em produção,Dupla costura,Cascão,Terminação deformada,Onda no fundo
MSE,1.707589,0.009202,5.308742,0.861620,0.545222,0.573459,10.168306,3.062033,90.072517,1.963028,73.574359,949.719687,188.710485,13.331667,0.211617,0.474247,0.755954,0.035293
R2,0.202061,0.000000,0.327954,-0.277658,-21.368095,-1.950270,-3.524274,0.027154,-0.105489,0.178649,0.036346,-0.077287,-0.128719,0.244665,-7.681731,-0.030972,-0.281278,-0.447938
